# Classificação Automática de Grãos de Trigo com Machine Learning

**Projeto:** Automatizando a Classificação de Grãos com Machine Learning  
**Dataset:** Seeds Dataset - UCI Machine Learning Repository  
**Metodologia:** CRISP-DM  
**Objetivo:** Classificar variedades de grãos de trigo (Kama, Rosa, Canadian) baseado em características físicas

## 1. Importação de Bibliotecas e Carregamento dos Dados

In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Configuração para visualizações
plt.style.use('default')
sns.set_palette("husl")

print("Bibliotecas importadas com sucesso")

In [ ]:
# Carregamento do dataset Seeds
# O dataset pode ser baixado de: https://archive.ics.uci.edu/dataset/236/seeds

# Definindo os nomes das colunas conforme documentação
column_names = [
    'area',
    'perimeter', 
    'compactness',
    'length_kernel',
    'width_kernel',
    'asymmetry_coefficient',
    'length_kernel_groove',
    'variety'
]

# Tentativa de carregar o dataset
try:
    # Carregando o arquivo (assumindo que está no mesmo diretório)
    df = pd.read_csv('seeds_dataset.txt', sep='\t', names=column_names)
    print(f"Dataset carregado: {df.shape[0]} amostras, {df.shape[1]} características")
except FileNotFoundError:
    # Criando dados sintéticos baseados nas características do dataset original
    print("Arquivo não encontrado. Criando dados sintéticos baseados no Seeds Dataset...")
    
    np.random.seed(42)
    n_samples = 210  # 70 amostras por classe
    
    # Características baseadas nas estatísticas do dataset original
    # Kama (classe 1)
    kama_data = {
        'area': np.random.normal(14.8, 1.2, 70),
        'perimeter': np.random.normal(14.2, 0.8, 70),
        'compactness': np.random.normal(0.871, 0.023, 70),
        'length_kernel': np.random.normal(5.76, 0.32, 70),
        'width_kernel': np.random.normal(3.31, 0.22, 70),
        'asymmetry_coefficient': np.random.normal(3.12, 0.89, 70),
        'length_kernel_groove': np.random.normal(5.22, 0.32, 70),
        'variety': [1] * 70
    }
    
    # Rosa (classe 2)
    rosa_data = {
        'area': np.random.normal(18.7, 1.8, 70),
        'perimeter': np.random.normal(16.1, 1.1, 70),
        'compactness': np.random.normal(0.886, 0.029, 70),
        'length_kernel': np.random.normal(6.17, 0.51, 70),
        'width_kernel': np.random.normal(3.58, 0.29, 70),
        'asymmetry_coefficient': np.random.normal(2.73, 0.81, 70),
        'length_kernel_groove': np.random.normal(5.99, 0.39, 70),
        'variety': [2] * 70
    }
    
    # Canadian (classe 3)
    canadian_data = {
        'area': np.random.normal(12.4, 1.1, 70),
        'perimeter': np.random.normal(13.2, 0.7, 70),
        'compactness': np.random.normal(0.849, 0.034, 70),
        'length_kernel': np.random.normal(5.22, 0.43, 70),
        'width_kernel': np.random.normal(2.87, 0.27, 70),
        'asymmetry_coefficient': np.random.normal(4.11, 1.18, 70),
        'length_kernel_groove': np.random.normal(4.63, 0.39, 70),
        'variety': [3] * 70
    }
    
    # Combinando os dados
    all_data = {}
    for key in kama_data.keys():
        all_data[key] = np.concatenate([kama_data[key], rosa_data[key], canadian_data[key]])
    
    df = pd.DataFrame(all_data)
    
    # Embaralhando os dados
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)
    
    print(f"Dados sintéticos criados: {df.shape[0]} amostras, {df.shape[1]} características")

# Exibindo informações básicas do dataset
print("\nPrimeiras 5 linhas do dataset:")
print(df.head())

print("\nInformações do dataset:")
print(df.info())

print("\nDistribuição das classes:")
print(df['variety'].value_counts().sort_index())

## 2. Análise Exploratória dos Dados

In [ ]:
# Estatísticas descritivas
print("Estatísticas Descritivas:")
print(df.describe())

# Verificação de valores ausentes
print("\nValores ausentes por coluna:")
print(df.isnull().sum())

# Mapeamento das variedades para nomes mais descritivos
variety_names = {1: 'Kama', 2: 'Rosa', 3: 'Canadian'}
df['variety_name'] = df['variety'].map(variety_names)

print("\nDistribuição das variedades:")
print(df['variety_name'].value_counts())

In [ ]:
# Visualização da distribuição das características
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()

features = ['area', 'perimeter', 'compactness', 'length_kernel', 
           'width_kernel', 'asymmetry_coefficient', 'length_kernel_groove']

for i, feature in enumerate(features):
    axes[i].hist(df[feature], bins=20, alpha=0.7, edgecolor='black')
    axes[i].set_title(f'Distribuição - {feature}')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Frequência')

# Removendo o subplot extra
axes[7].remove()

plt.tight_layout()
plt.show()

In [ ]:
# Boxplots para identificar outliers
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.ravel()

for i, feature in enumerate(features):
    axes[i].boxplot(df[feature])
    axes[i].set_title(f'Boxplot - {feature}')
    axes[i].set_ylabel(feature)

axes[7].remove()
plt.tight_layout()
plt.show()

In [ ]:
# Análise de correlação entre características
correlation_matrix = df[features].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, fmt='.2f')
plt.title('Matriz de Correlação entre Características')
plt.tight_layout()
plt.show()

# Identificando correlações altas
print("Correlações altas (> 0.7):")
high_corr = np.where(np.abs(correlation_matrix) > 0.7)
for i, j in zip(high_corr[0], high_corr[1]):
    if i < j:  # Evitar duplicatas
        print(f"{features[i]} - {features[j]}: {correlation_matrix.iloc[i, j]:.3f}")

In [ ]:
# Gráficos de dispersão para visualizar separabilidade das classes
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

# Pares de características mais relevantes
feature_pairs = [
    ('area', 'perimeter'),
    ('length_kernel', 'width_kernel'),
    ('compactness', 'asymmetry_coefficient'),
    ('area', 'length_kernel'),
    ('perimeter', 'width_kernel'),
    ('length_kernel_groove', 'asymmetry_coefficient')
]

colors = ['red', 'blue', 'green']
varieties = [1, 2, 3]
variety_labels = ['Kama', 'Rosa', 'Canadian']

for i, (x_feature, y_feature) in enumerate(feature_pairs):
    for j, variety in enumerate(varieties):
        subset = df[df['variety'] == variety]
        axes[i].scatter(subset[x_feature], subset[y_feature], 
                       c=colors[j], label=variety_labels[j], alpha=0.6)
    
    axes[i].set_xlabel(x_feature)
    axes[i].set_ylabel(y_feature)
    axes[i].set_title(f'{x_feature} vs {y_feature}')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Pré-processamento dos Dados

In [ ]:
# Preparação dos dados para modelagem
X = df[features].copy()
y = df['variety'].copy()

print(f"Formato dos dados: X = {X.shape}, y = {y.shape}")

# Verificação da necessidade de normalização
print("\nEstatísticas das características (antes da normalização):")
print(X.describe())

# Análise da escala das variáveis
print("\nRanges das características:")
for feature in features:
    min_val = X[feature].min()
    max_val = X[feature].max()
    range_val = max_val - min_val
    print(f"{feature}: [{min_val:.3f}, {max_val:.3f}] - Range: {range_val:.3f}")

In [ ]:
# Divisão dos dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"Conjunto de treinamento: {X_train.shape[0]} amostras")
print(f"Conjunto de teste: {X_test.shape[0]} amostras")

# Verificando a distribuição das classes nos conjuntos
print("\nDistribuição das classes no conjunto de treinamento:")
print(y_train.value_counts().sort_index())

print("\nDistribuição das classes no conjunto de teste:")
print(y_test.value_counts().sort_index())

In [ ]:
# Normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convertendo de volta para DataFrame para facilitar análise
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=features)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=features)

print("Estatísticas após normalização (conjunto de treinamento):")
print(X_train_scaled_df.describe())

print("\nNormalização aplicada com sucesso")
print("Média próxima de 0 e desvio padrão próximo de 1 confirmam a padronização")

## 4. Implementação e Comparação de Algoritmos de Classificação

In [ ]:
# Definindo os modelos de classificação
models = {
    'KNN': KNeighborsClassifier(),
    'SVM': SVC(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Naive Bayes': GaussianNB(),
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000)
}

# Dicionário para armazenar resultados
results = {}

print("Treinando e avaliando modelos...\n")

# Treinamento e avaliação de cada modelo
for name, model in models.items():
    print(f"Treinando {name}...")
    
    # Treinamento
    model.fit(X_train_scaled, y_train)
    
    # Predições
    y_pred = model.predict(X_test_scaled)
    
    # Métricas
    accuracy = accuracy_score(y_test, y_pred)
    
    # Validação cruzada
    cv_scores = cross_val_score(model, X_train_scaled, y_train, cv=5)
    
    # Armazenando resultados
    results[name] = {
        'model': model,
        'accuracy': accuracy,
        'cv_mean': cv_scores.mean(),
        'cv_std': cv_scores.std(),
        'predictions': y_pred
    }
    
    print(f"Acurácia no teste: {accuracy:.4f}")
    print(f"Validação cruzada: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
    print("-" * 50)

In [ ]:
# Comparação visual dos resultados
model_names = list(results.keys())
accuracies = [results[name]['accuracy'] for name in model_names]
cv_means = [results[name]['cv_mean'] for name in model_names]
cv_stds = [results[name]['cv_std'] for name in model_names]

# Gráfico de barras comparativo
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Acurácia no teste
bars1 = ax1.bar(model_names, accuracies, color='skyblue', alpha=0.7)
ax1.set_title('Acurácia no Conjunto de Teste')
ax1.set_ylabel('Acurácia')
ax1.set_ylim(0, 1)
ax1.tick_params(axis='x', rotation=45)

# Adicionando valores nas barras
for bar, acc in zip(bars1, accuracies):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{acc:.3f}', ha='center', va='bottom')

# Validação cruzada
bars2 = ax2.bar(model_names, cv_means, yerr=cv_stds, 
                color='lightcoral', alpha=0.7, capsize=5)
ax2.set_title('Validação Cruzada (5-fold)')
ax2.set_ylabel('Acurácia Média')
ax2.set_ylim(0, 1)
ax2.tick_params(axis='x', rotation=45)

# Adicionando valores nas barras
for bar, mean_val in zip(bars2, cv_means):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01, 
             f'{mean_val:.3f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

# Tabela resumo dos resultados
results_df = pd.DataFrame({
    'Modelo': model_names,
    'Acurácia Teste': [f"{acc:.4f}" for acc in accuracies],
    'CV Média': [f"{mean:.4f}" for mean in cv_means],
    'CV Desvio': [f"{std:.4f}" for std in cv_stds]
})

print("\nResumo dos Resultados:")
print(results_df.to_string(index=False))

In [ ]:
# Relatórios de classificação detalhados
variety_labels = ['Kama', 'Rosa', 'Canadian']

for name in model_names:
    print(f"\n{'='*60}")
    print(f"RELATÓRIO DETALHADO - {name}")
    print(f"{'='*60}")
    
    y_pred = results[name]['predictions']
    
    # Relatório de classificação
    print("\nRelatório de Classificação:")
    print(classification_report(y_test, y_pred, target_names=variety_labels))
    
    # Matriz de confusão
    cm = confusion_matrix(y_test, y_pred)
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=variety_labels, yticklabels=variety_labels)
    plt.title(f'Matriz de Confusão - {name}')
    plt.xlabel('Predito')
    plt.ylabel('Real')
    plt.show()

## 5. Otimização de Hiperparâmetros

In [ ]:
# Definindo os hiperparâmetros para otimização
param_grids = {
    'KNN': {
        'n_neighbors': [3, 5, 7, 9, 11],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    },
    'SVM': {
        'C': [0.1, 1, 10, 100],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto']
    },
    'Random Forest': {
        'n_estimators': [50, 100, 200],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10]
    },
    'Logistic Regression': {
        'C': [0.1, 1, 10, 100],
        'solver': ['liblinear', 'lbfgs'],
        'penalty': ['l2']
    }
}

# Naive Bayes não tem hiperparâmetros significativos para otimizar
optimized_results = {}

print("Iniciando otimização de hiperparâmetros...\n")

for name in param_grids.keys():
    print(f"Otimizando {name}...")
    
    # Grid Search
    grid_search = GridSearchCV(
        models[name], 
        param_grids[name], 
        cv=5, 
        scoring='accuracy',
        n_jobs=-1
    )
    
    grid_search.fit(X_train_scaled, y_train)
    
    # Melhor modelo
    best_model = grid_search.best_estimator_
    
    # Predições com o modelo otimizado
    y_pred_optimized = best_model.predict(X_test_scaled)
    accuracy_optimized = accuracy_score(y_test, y_pred_optimized)
    
    # Validação cruzada do modelo otimizado
    cv_scores_optimized = cross_val_score(best_model, X_train_scaled, y_train, cv=5)
    
    optimized_results[name] = {
        'best_model': best_model,
        'best_params': grid_search.best_params_,
        'best_cv_score': grid_search.best_score_,
        'test_accuracy': accuracy_optimized,
        'cv_mean': cv_scores_optimized.mean(),
        'cv_std': cv_scores_optimized.std(),
        'predictions': y_pred_optimized
    }
    
    print(f"Melhores parâmetros: {grid_search.best_params_}")
    print(f"Melhor score CV: {grid_search.best_score_:.4f}")
    print(f"Acurácia no teste: {accuracy_optimized:.4f}")
    
    # Comparação com modelo original
    improvement = accuracy_optimized - results[name]['accuracy']
    print(f"Melhoria: {improvement:+.4f}")
    print("-" * 60)

# Adicionando Naive Bayes (sem otimização)
optimized_results['Naive Bayes'] = {
    'best_model': results['Naive Bayes']['model'],
    'best_params': 'Não aplicável',
    'test_accuracy': results['Naive Bayes']['accuracy'],
    'cv_mean': results['Naive Bayes']['cv_mean'],
    'cv_std': results['Naive Bayes']['cv_std'],
    'predictions': results['Naive Bayes']['predictions']
}

In [ ]:
# Comparação antes e depois da otimização
comparison_data = []

for name in model_names:
    original_acc = results[name]['accuracy']
    
    if name in optimized_results:
        optimized_acc = optimized_results[name]['test_accuracy']
        improvement = optimized_acc - original_acc
    else:
        optimized_acc = original_acc
        improvement = 0
    
    comparison_data.append({
        'Modelo': name,
        'Acurácia Original': f"{original_acc:.4f}",
        'Acurácia Otimizada': f"{optimized_acc:.4f}",
        'Melhoria': f"{improvement:+.4f}"
    })

comparison_df = pd.DataFrame(comparison_data)
print("\nComparação: Original vs Otimizado")
print("=" * 50)
print(comparison_df.to_string(index=False))

# Visualização da comparação
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(model_names))
width = 0.35

original_accs = [results[name]['accuracy'] for name in model_names]
optimized_accs = [optimized_results.get(name, results[name])['test_accuracy'] for name in model_names]

bars1 = ax.bar(x - width/2, original_accs, width, label='Original', alpha=0.7)
bars2 = ax.bar(x + width/2, optimized_accs, width, label='Otimizado', alpha=0.7)

ax.set_xlabel('Modelos')
ax.set_ylabel('Acurácia')
ax.set_title('Comparação: Modelos Originais vs Otimizados')
ax.set_xticks(x)
ax.set_xticklabels(model_names, rotation=45)
ax.legend()
ax.set_ylim(0, 1)

# Adicionando valores nas barras
for bars in [bars1, bars2]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                f'{height:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

## 6. Análise e Interpretação dos Resultados

In [ ]:
# Identificando o melhor modelo
best_model_name = max(optimized_results.keys(), 
                     key=lambda x: optimized_results[x]['test_accuracy'])
best_model_info = optimized_results[best_model_name]

print(f"MELHOR MODELO: {best_model_name}")
print(f"Acurácia no teste: {best_model_info['test_accuracy']:.4f}")
print(f"Validação cruzada: {best_model_info['cv_mean']:.4f} (+/- {best_model_info['cv_std']*2:.4f})")

if 'best_params' in best_model_info and best_model_info['best_params'] != 'Não aplicável':
    print(f"Melhores parâmetros: {best_model_info['best_params']}")

# Análise detalhada do melhor modelo
print(f"\n{'='*60}")
print(f"ANÁLISE DETALHADA DO MELHOR MODELO: {best_model_name}")
print(f"{'='*60}")

best_predictions = best_model_info['predictions']
print("\nRelatório de Classificação:")
print(classification_report(y_test, best_predictions, target_names=variety_labels))

# Matriz de confusão do melhor modelo
cm_best = confusion_matrix(y_test, best_predictions)

plt.figure(figsize=(8, 6))
sns.heatmap(cm_best, annot=True, fmt='d', cmap='Greens', 
            xticklabels=variety_labels, yticklabels=variety_labels)
plt.title(f'Matriz de Confusão - {best_model_name} (Melhor Modelo)')
plt.xlabel('Predito')
plt.ylabel('Real')
plt.show()

# Análise de erros
errors = y_test != best_predictions
error_count = errors.sum()
total_count = len(y_test)

print(f"\nAnálise de Erros:")
print(f"Total de erros: {error_count} de {total_count} amostras")
print(f"Taxa de erro: {error_count/total_count:.4f}")

if error_count > 0:
    print("\nAmostras classificadas incorretamente:")
    error_indices = np.where(errors)[0]
    for idx in error_indices:
        actual = y_test.iloc[idx]
        predicted = best_predictions[idx]
        print(f"Amostra {idx}: Real = {variety_names[actual]}, Predito = {variety_names[predicted]}")

In [ ]:
# Análise da importância das características (para modelos que suportam)
if best_model_name == 'Random Forest':
    feature_importance = best_model_info['best_model'].feature_importances_
    
    # Criando DataFrame para visualização
    importance_df = pd.DataFrame({
        'Característica': features,
        'Importância': feature_importance
    }).sort_values('Importância', ascending=False)
    
    print("\nImportância das Características (Random Forest):")
    print(importance_df.to_string(index=False))
    
    # Visualização
    plt.figure(figsize=(10, 6))
    bars = plt.bar(importance_df['Característica'], importance_df['Importância'])
    plt.title('Importância das Características - Random Forest')
    plt.xlabel('Características')
    plt.ylabel('Importância')
    plt.xticks(rotation=45)
    
    # Adicionando valores nas barras
    for bar, importance in zip(bars, importance_df['Importância']):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
                f'{importance:.3f}', ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

elif best_model_name == 'Logistic Regression':
    # Coeficientes da regressão logística
    coefficients = best_model_info['best_model'].coef_
    
    print("\nCoeficientes da Regressão Logística:")
    for i, variety in enumerate(['Kama', 'Rosa', 'Canadian']):
        print(f"\n{variety}:")
        coef_df = pd.DataFrame({
            'Característica': features,
            'Coeficiente': coefficients[i]
        }).sort_values('Coeficiente', key=abs, ascending=False)
        print(coef_df.to_string(index=False))

In [ ]:
# Insights e conclusões
print("\n" + "="*80)
print("INSIGHTS E CONCLUSÕES")
print("="*80)

print("\n1. PERFORMANCE DOS MODELOS:")
print("-" * 40)

# Ranking dos modelos
model_ranking = sorted(optimized_results.items(), 
                      key=lambda x: x[1]['test_accuracy'], reverse=True)

for i, (name, info) in enumerate(model_ranking, 1):
    print(f"{i}. {name}: {info['test_accuracy']:.4f}")

print("\n2. ANÁLISE DAS CARACTERÍSTICAS:")
print("-" * 40)

# Características mais correlacionadas
high_corr_pairs = []
for i in range(len(features)):
    for j in range(i+1, len(features)):
        corr_val = correlation_matrix.iloc[i, j]
        if abs(corr_val) > 0.7:
            high_corr_pairs.append((features[i], features[j], corr_val))

if high_corr_pairs:
    print("Características altamente correlacionadas:")
    for feat1, feat2, corr in high_corr_pairs:
        print(f"  {feat1} - {feat2}: {corr:.3f}")
else:
    print("Não foram encontradas correlações muito altas entre características.")

print("\n3. SEPARABILIDADE DAS CLASSES:")
print("-" * 40)

# Análise da distribuição das classes
class_stats = df.groupby('variety')[features].mean()
print("Médias das características por classe:")
print(class_stats.round(3))

print("\n4. RECOMENDAÇÕES PARA APLICAÇÃO PRÁTICA:")
print("-" * 40)

best_accuracy = best_model_info['test_accuracy']

if best_accuracy >= 0.95:
    print("• Excelente performance - Modelo pronto para produção")
elif best_accuracy >= 0.90:
    print("• Boa performance - Adequado para uso com monitoramento")
elif best_accuracy >= 0.85:
    print("• Performance moderada - Considerar mais dados ou features")
else:
    print("• Performance baixa - Necessita melhorias significativas")

print(f"• Modelo recomendado: {best_model_name}")
print(f"• Acurácia esperada: {best_accuracy:.1%}")
print("• Aplicação: Classificação automática de grãos de trigo")
print("• Benefícios: Redução de tempo e erro humano na classificação")

print("\n5. LIMITAÇÕES E CONSIDERAÇÕES:")
print("-" * 40)
print("• Dataset relativamente pequeno (210 amostras)")
print("• Necessidade de validação com dados de diferentes origens")
print("• Importância da calibração dos equipamentos de medição")
print("• Consideração de fatores ambientais na coleta")

print("\n" + "="*80)
print("PROJETO CONCLUÍDO COM SUCESSO")
print("Metodologia CRISP-DM aplicada integralmente")
print("Sistema de classificação automática desenvolvido e validado")
print("="*80)